# API analytics exercise to extract active user data

In [6]:
import dtlpy as dl #Dataloop's Python SDK Package
import requests #library for API requests
import time
# remember to log in to Dataloop before doing anything.

# dl.login() #here is the -log in from above-

if dl.token_expired(): # This line checks if you are logged in to Dataloop, and if not, it logs you in
    dl.login()         # If it doesn't work, use the simple -log in from above-

## A function that gets the active Users and returns the information as a hash using a POST request

In [13]:

def get_active_users(project, start_time, end_time=None):
    report = list()
    page_size = 2
    page = 0
    has_next = True
    while has_next:
        payload = {
            "startTime": start_time,
            "endTime": end_time,
            "context": {
                "projectId": [project.id],
            },
            "measures": [{"measureType": "activeUsers",
                          "page": page,
                          "pageSize": page_size,
                          "sortDirection": "descending"}]
        }

        samples = requests.post(dl.environment() + '/analytics/query',
                                headers=dl.client_api.auth,
                                json=payload)
        try:
            samples = samples.json()
        except KeyError as r:
            print('analytics report: Error: {}'.format(r))
            return

        if len(samples) != 0 and 'response' in samples[0]:
            report += samples[0]['response']
            has_next = samples[0]['hasNext']
            page += 1
        else:
            has_next = False
    return get_user_info_by_hash(report)


def get_user_info_by_hash(hash_tokens):
    samples = requests.post(dl.environment() + '/users/hash',
                            headers=dl.client_api.auth,
                            json=hash_tokens)
    try:
        samples = samples.json()
    except KeyError as r:
        print('analytics report: Error: {}'.format(r))
        return
    return samples



# A function that gets a all User information (report) as a hash. It uses the get_active_users() function above to get the users and then it also gets the user information

In [8]:

def get_report(project: dl.Project, start_time: int, end_time: int = None):
    """
    generate list of activities for all annotators in all tasks between dates
    :param project: project to execute the report
    :param start_time: MS since the epoch, in UTC.
    :param end_time: MS since the epoch, in UTC.
    :return: report, active_users
    """
    report = list()
    page_size = 100
    page = 0
    has_next = True
    while has_next:
        payload = {
            "startTime": start_time,
            "endTime": end_time,
            "context": {
                "projectId": [project.id],
            },
            "measures": [{"measureType": "userStatsActivityTimeByField",
                          "page": page,
                          "pageSize": page_size,
                          "params": {
                              "groupByFields": ["taskId", "itemId", "userId"]
                          },
                          "sortDirection": "descending"}]
        }
        samples = requests.post(dl.environment() + '/analytics/query',
                                headers=dl.client_api.auth,
                                json=payload)
        try:
            samples = samples.json()
        except KeyError as r:
            print('analytics report: Error: {}'.format(r))
            return
        if len(samples) != 0 and 'response' in samples[0]:
            report += samples[0]['response']
            has_next = samples[0]['hasNext']
            page += 1
        else:
            has_next = False
    active_users_list = get_active_users(project=project, start_time=start_time, end_time=None)
    active_users = dict()
    for active_user in active_users_list:
        active_users[active_user['hash']] = active_user

    return report, active_users



## The main function where we define the time from which we want to Querry(start_time), select the project we want to run analytics on and then print everything we extracted on screen



In [ ]:
def main():
    dl.setenv("rc")
    # dl.login()
    now = int(time.time() * 1000)
    day = 60*60*24*1000
    start_time = now - day * 30
    # the start_time example above shows how you can define the starting time for the analytics as 30 days ag
    # if you want to set the start time to 90 days ago, replace the 30 with 90
    
    project = dl.projects.get(project_name="My-First-Project1")#change with your own project's name
    
    #we set the start time = when the project was created (Querry all of the Project's timeline)
    start_time = project.created_at 
    
    report, active_user = get_report(project=project, start_time=start_time) 
    #printing the extracted information
    for line in report:
        print("taskId: {}, itemId: {} user: {}, activityStatus: {}, duration {} ".format(
            line['taskId'],
            line['itemId'],
            active_user[line['userId']]['username'],
            line['activityStatus'],
            line['duration']))

#after we define the main, we simple execute it, as seen below
if __name__ == "__main__":
    main()
